In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import os.path as osp
import sys
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.ar_model import AutoReg

sys.path.insert(0,'src')

In [2]:
from data_funcs import plot_data, load_and_fix_data, rmse

In [3]:
# Read data at one location
# all RAWS observations
dat={}
dat.update(load_and_fix_data('data/raws_dat.pickle'))
dat1 = dat['CPTC2_2023-05-01'] # restrict to one station for visualization

After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan values
After fixing, remained 0 nan

In [4]:
h2=int(20*24) # length of training period

## AR Model

For a model with $K$ time lags and $P$ other covariates, the mathematical form is:

$$
y_t = \beta_0 + \sum_{k=1}^K \beta_k y_{t-k} + \sum_{i=1}^P\alpha_j x_{j, t} +\epsilon_t
$$

We will conduct a variable selection procedure with normal AR packages and an unweighted loss function. Then, this model will be recreated with `sklearn.Linear_Regression` in order to weight it. The variable selection procedure will compare the out-of-sample RMSE for several different time lags  and a standard set of FMDA covariates.

Many models include a time trend, or adding a term $\beta t$, which would model a constant upward or downward trend. Moving average components are often used in time series modeling, extending the AR concept to ARIMA. For theoretical reasons, we are only considering autoregressive terms and not moving averages. We are using several other covariates, and the variable selection procedure could get cumbersome and lead to overfitting. Additionally, autoregressive covariate terms theoretically resemble finite-difference methods for differential equations, which are traditionally used in the Earth sciences. We don't see a corresponding scientific justification for moving average terms. For FMDA data, we will choose not to consider such time trend terms, as this is likely to chase noice in the training set. Additionally, we will include the hour of the day (1-24) to account for the daily cyclical patterns in fuel moisture. In the AR literature, this would be a "seasonal" affect, and would correspond to SARIMA modeling.

We will initially consider time lags of 1, 2, 5, and 24. 

In [5]:
# Helper Functions for AR Model

def build_design(d):
    # Build design matrix 
    # Inputs:
    # d: FMDA dictionary
    # Returns: matrix
    hours = len(d['fm'])
    hour = np.resize(range(1, 24), hours) # repeat 1-24 for each response value (times here aren't real)
    
    XX = np.column_stack((hour, d['Ed'], d['rain'], 
                          d['wind_speed']))
    return XX
    
def train_AR(fm, X, lag, h2 = int(20*24)):
    fmtr = fm[0:h2]
    ar = AutoReg(fmtr, lags=lag, trend="c", exog = X[0:h2]).fit()
    return ar
def predict_AR(ar, X, hours, h2 = int(20*24)):
    return ar.model.predict(ar.params, start=h2, end=hours-1, exog_oos=X[h2:hours], dynamic=False)
    
# fmtr = dat['fm'][0:h2] # response var over training period
# fmte = dat['fm'][h2:hours] # response var over forecast period

In [6]:
XX = build_design(dat1)
fm = dat1['fm']
hours = len(fm)
fmte = fmte = dat1['fm'][h2:hours] # response var over forecast period
ar1 = train_AR(fm, XX, lag=1)
ar2 = train_AR(fm, XX, lag=2)
ar5 = train_AR(fm, XX, lag=5)
ar24 = train_AR(fm, XX, lag=24)

In [7]:
# Print AIC values for quick comparison
print(f'AR Lag 1 AIC: {np.round(ar1.aic, 5)}')
print(f'AR Lag 2 AIC: {np.round(ar2.aic, 5)}')
print(f'AR Lag 5 AIC: {np.round(ar5.aic, 5)}')
print(f'AR Lag 24 AIC: {np.round(ar24.aic, 5)}')

AR Lag 1 AIC: 1493.62957
AR Lag 2 AIC: 1387.88801
AR Lag 5 AIC: 1378.99822
AR Lag 24 AIC: 1275.89443


In [8]:
# Compare Forecast Accuracy
preds1 = predict_AR(ar1, XX, hours)
preds2 = predict_AR(ar2, XX, hours)
preds5 = predict_AR(ar5, XX, hours)
preds24 = predict_AR(ar24, XX, hours)

In [9]:
# Print forecast RMSE values 
print(f'AR Lag 1 RMSE: {np.round(rmse(preds1, fmte), 5)}')
print(f'AR Lag 2 RMSE: {np.round(rmse(preds2, fmte), 5)}')
print(f'AR Lag 5 RMSE: {np.round(rmse(preds5, fmte), 5)}')
print(f'AR Lag 24 RMSE: {np.round(rmse(preds24, fmte), 5)}')

AR Lag 1 RMSE: 3.59131
AR Lag 2 RMSE: 3.02873
AR Lag 5 RMSE: 3.10031
AR Lag 24 RMSE: 3.63993


For the one dataset considered, the Lag 2 model has the lowest forecast error. We will now deploy this procedure on additional locations to arrive at a final model specification. Hastie and Tibshirani propose a 1-standard-error rule, where you opt for the simplest model that is within 1 standard error of a given accuracy metric for the most accurate model. This is a very conservative approach by design.

## Weighted AR Model

In [ ]:
# Helper Functions
def build_lags(v, lags):
    "v: data vector to lag"
    "lags: list of integers"
    
    X = pd.DataFrame({'x': v})
    for l in lags:
        X[f"lag{l}"] = X['x'].shift(l)
    X = X.drop(['x'], axis=1)
    X = X.dropna().to_numpy()
    return X

def predict_ar(m, K, f, XX, ts):
    "m: model object"
    "K: time lag terms in m"
    "f: observed"
    "XX: covariate matrix"
    "ts: number of time steps to forecast"
    
    preds = np.zeros(ts) # initialize array of forecasts for return value
    
    Xtemp = np.column_stack((np.flip(f[-K:]).reshape(1, K), XX.loc[0:0])) # model matrix with last fitted value

    preds[0]=m.predict(Xtemp)

    # Loop through remaining time steps and predict using last value
    for i in range(1, ts):
        if i < K: # build lags using training data if necessary
            x = np.concatenate((f[-(K-i):], preds[0:i]))
        else: 
            x = preds[(i-K):i]
        x = np.flip(x)
        Xtemp = np.column_stack((x.reshape(1, K), XX.loc[i:i]))
        # Xtemp = preds[i-1].reshape(1, 1) # join with time index
        preds[i]=m.predict(Xtemp)
    
    return preds

def build_ar(dat, lags, hours = 720, h2 = 480):
    # Input:
    # dat: dictionary of fmda data
    # lags: (int) time lags to model
    
    # Return dictionary
    mod={'h2': h2, 'hours':hours}

    # Time params
    h = np.arange(0, hours)
    hour = np.resize(range(0, 23), hours) # repeat 0-23 starting at time 0, not necessarily lined up with actual time of day
    
    # Build training matrix
    X = build_lags(dat['fm'][0:h2], lags = np.arange(1, lags+1))
    X = pd.DataFrame(X)
    X['t'] = h[lags:h2].tolist()
    X['hour'] = hour[lags:h2].tolist()
    X['rain'] = dat['rain'][lags:h2].tolist()
    X['Ed'] = dat['Ed'][lags:h2].tolist()
    X['wind_speed'] = dat['wind_speed'][lags:h2].tolist()
    X = X.to_numpy()
    
    mod['train']=X
    
    # Fit model
    mod["m"] = LinearRegression().fit(X, dat['fm'][lags:h2])
    mod['fits'] = mod["m"].predict(X)

    # # Set up prediction matrix
    X2 = pd.DataFrame({'t': h[h2:hours].tolist(), 'hour': hour[h2:hours].tolist(), 'rain': dat['rain'][h2:hours].tolist(), 
                       'Ed': dat['Ed'][h2:hours].tolist(), 'wind_speed': dat['wind_speed'][h2:hours].tolist()})
    mod['test']=X2
    
    # mod['preds']=predict_ar(mod['m'], lags, dat['fm'], X2, len(dat['fm'])-mod1['h2'])
    
    return mod

In [ ]:
mod1 = build_ar(dat, 1)